# Bloque 9: Visualización Espacial y Operaciones 🗺️

En este bloque aprenderás a:
✅ Crear mapas temáticos y avanzados con GeoPandas.  
✅ Combinar capas de datos espaciales.  
✅ Realizar operaciones espaciales básicas (intersección, unión, contención).  
✅ Crear mapas interactivos con Folium.

💡 *Objetivo:* producir mapas informativos que integren visualización, análisis y contexto geográfico.


2. Instalación y carga de librerías

In [1]:
!pip install geopandas contextily folium matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.7 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import contextily as ctx
import folium
import matplotlib.pyplot as plt


3. Cargar el dataset geoespacial

In [6]:
# Cargar el GeoDataFrame creado en el bloque anterior
gdf = gpd.read_file('/content/gbif_dasypus_mapa.geojson')
gdf.head()


DataSourceError: Failed to read GeoJSON data; At line 639, character 837: Unterminated object

4. Revisión del CRS y proyección

In [ ]:
gdf.crs


In [ ]:
# Asegurar proyección en Mercator para mapas base
gdf = gdf.to_crs(epsg=3857)


5. Mapa básico con Contextily

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, color='seagreen', markersize=20, alpha=0.7, edgecolor='black')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=gdf.crs.to_string())
ax.set_title('Distribución geográfica de Dasypus novemcinctus')
ax.set_axis_off()
plt.show()


Ejercicio:
Prueba cambiar el mapa base a ctx.providers.CartoDB.Voyager o ctx.providers.OpenStreetMap.Mapnik

6. Mapas temáticos avanzados

In [ ]:
# 1. Contar registros por countryCode
registros_pais = gdf.groupby('countryCode').size().reset_index(name='count')

# 2. Cargar límites de países del mundo (Natural Earth 1:110m)
world = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")

# 3. Unir usando ISO_A2 (códigos de 2 letras, en mayúsculas)
world_gbif = world.merge(registros_pais, how='left', left_on='ISO_A2', right_on='countryCode')

# 4. Rellenar los conteos faltantes con 0
world_gbif['count'] = world_gbif['count'].fillna(0)

# 5. Reproyectar a EPSG:3857 (Web Mercator) para usar con contextily
world_gbif = world_gbif.to_crs(epsg=3857)

# 6. Crear el gráfico
fig, ax = plt.subplots(figsize=(12, 6))

# Dibujar el mapa coroplético
world_gbif.plot(
    column='count',
    cmap='YlGnBu',
    legend=True,
    edgecolor='black',
    linewidth=0.5,
    ax=ax,
    missing_kwds={"color": "lightgrey"}
)

# Agregar mapa base (CartoDB Positron)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=world_gbif.crs.to_string())

# Forzar vista global (opcional pero recomendado para evitar zoom automático)
ax.set_xlim(-20037508.34, 20037508.34)
ax.set_ylim(-20037508.34, 20037508.34)

# Título y limpieza visual
plt.title('Número de registros por país')
plt.axis('off')
plt.tight_layout()
plt.show()

Interpretación:
¿Dónde hay más registros? ¿En qué regiones faltan datos?

7. Operaciones espaciales básicas

In [ ]:
# 1. Cargar el mapa mundial desde Natural Earth (1:110m cultural)
world = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")

# 2. Reproyectar a Web Mercator (EPSG:3857) — pero solo después de filtrar
#    (mejor práctica: hacer operaciones espaciales en CRS geográfico si tus datos están en lat/lon)

# Asegúrate de que tu gdf tenga un CRS definido (probablemente EPSG:4326 si son coordenadas decimales)
if gdf.crs is None:
    gdf = gdf.set_crs("EPSG:4326")

# 3. Filtrar México (usando el nombre en inglés, tal como aparece en Natural Earth)
mexico = world[world['NAME'] == 'Mexico']  # ¡Importante: 'NAME', no 'name'!

# 4. Asegurar que ambos GeoDataFrames tengan el mismo CRS para la intersección
mexico = mexico.to_crs(gdf.crs)

# 5. Realizar la intersección espacial: puntos dentro de México
gdf_mexico = gpd.sjoin(gdf, mexico, how='inner', predicate='within')

# 6. Resultados
print(f"Registros totales: {len(gdf)}")
print(f"Registros dentro de México: {len(gdf_mexico)}")

*Ejercicio:
Repite el análisis para otro país o región.
¿Dónde se concentra la mayor parte de los registros?*

8. Visualización interactiva con Folium

In [ ]:
# 1. Asegurar que gdf tiene un CRS definido (asumimos EPSG:4326 si son coordenadas decimales)
if gdf.crs is None:
    gdf = gdf.set_crs("EPSG:4326")

# 2. Convertir a WGS84 (EPSG:4326) si no está ya
gdf_wgs84 = gdf.to_crs("EPSG:4326")

# 3. Eliminar geometrías inválidas o vacías (opcional pero recomendado)
gdf_wgs84 = gdf_wgs84[gdf_wgs84.geometry.notnull()]
gdf_wgs84 = gdf_wgs84[gdf_wgs84.is_valid]

# 4. Calcular centro del mapa (con manejo de errores)
try:
    center_lat = gdf_wgs84.geometry.y.mean()
    center_lon = gdf_wgs84.geometry.x.mean()
except Exception as e:
    # Si falla, usar un centro predeterminado (ej. ecuador)
    center_lat, center_lon = 10, -30

# 5. Crear mapa
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=3,
    tiles='CartoDB positron'
)

# 6. Agregar puntos usando FastMarkerCluster (opcional) o FeatureGroup para mejor rendimiento
# Opción A: Si son pocos puntos (<1000), usar CircleMarker en un FeatureGroup
feature_group = folium.FeatureGroup(name="Registros")

for _, row in gdf_wgs84.iterrows():
    popup_text = f"Especie: {row.get('scientificName', 'N/A')}<br>País: {row.get('countryCode', 'N/A')}"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='green',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(feature_group)

feature_group.add_to(m)

# 7. Opción B (recomendada si hay >1000 puntos): usar MarkerCluster o simplemente GeoJSON
# Alternativa eficiente (descomenta si tienes muchos puntos):
"""
folium.GeoJson(
    gdf_wgs84[['scientificName', 'countryCode', 'geometry']].__geo_interface__,
    name='Registros',
    marker=folium.CircleMarker(
        radius=4, color='green', fill=True, fill_opacity=0.6
    ),
    tooltip=folium.GeoJsonTooltip(fields=['scientificName', 'countryCode'], aliases=['Especie:', 'País:'])
).add_to(m)
"""

# Mostrar mapa
m

*Reto:
Modifica el color de los puntos según country o basisOfRecord.
Puedes usar folium.plugins.MarkerCluster para agrupar muchos puntos.*

# 🧠 Mini Reto: Mapa comparativo de especies

1. Descarga o carga los registros de **otra especie** desde GBIF.  
2. Crea un GeoDataFrame y un mapa base.  
3. Visualiza ambas especies en el mismo mapa (colores diferentes).  
4. Agrega un mapa interactivo con Folium mostrando las dos distribuciones.

💬 *Preguntas:*
- ¿Las distribuciones se solapan o son distintas?
- ¿Qué factores ecológicos podrían explicar esta diferencia?


Ejemplo de superposición de dos especies:

In [ ]:
gdf2 = gpd.read_file('gbif_Leopardus_pardalis.geojson').to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(10,6))
gdf.plot(ax=ax, color='seagreen', markersize=20, label='Dasypus novemcinctus', alpha=0.6)
gdf2.plot(ax=ax, color='orange', markersize=20, label='Leopardus pardalis', alpha=0.6)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
plt.legend()
plt.title('Comparación espacial entre dos especies')
ax.set_axis_off()
plt.show()


10. Exportar resultados

In [ ]:
# Exportar a archivos para GIS o compartición
gdf_mexico.to_file('gbif_dasypus_mexico.geojson', driver='GeoJSON')
print("Archivo exportado: gbif_dasypus_mexico.geojson")


En este bloque aprendiste a:
✅ Crear mapas temáticos avanzados con GeoPandas y Contextily.  
✅ Realizar operaciones espaciales básicas (intersección, selección).  
✅ Integrar tus datos con mapas base reales.  
✅ Crear mapas interactivos con Folium para exploración visual.

🚀 En el siguiente bloque (Bloque 10, opcional) realizarás un **mini proyecto de análisis espacial completo**, integrando los pasos desde la descarga de datos hasta la visualización final.
